# **LLM Juge - RAG : April 2025**

Making an LLM JUge to co;pare the quality of the answers given by a RAG.
The RAg is based on GPT 4.o
The Juge is based on Llama3


**Import** **environmement**

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # Revenir à "0" plus tard
os.environ['HF_HOME']="/work/LLM_models/HF_HOME"

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install langchain_community

In [ ]:
!pip install ragatouille
!pip install colbert

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade ragatouille
!pip install --upgrade colbert

In [ ]:
!pip uninstall colbert ragatouille -y
!pip install ragatouille

Found existing installation: Colbert 0.40
Uninstalling Colbert-0.40:
  Successfully uninstalled Colbert-0.40
Found existing installation: RAGatouille 0.0.9
Uninstalling RAGatouille-0.0.9:
  Successfully uninstalled RAGatouille-0.0.9
  Using cached RAGatouille-0.0.9-py3-none-any.whl.metadata (28 kB)
Using cached RAGatouille-0.0.9-py3-none-any.whl (45 kB)


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from langchain_community.document_loaders import BSHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_transformers import Html2TextTransformer
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
#===================================================================================================================
# Message history management
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
#===================================================================================================================
from langchain.prompts import PromptTemplate
from langchain.vectorstores.utils import filter_complex_metadata

#### ERROR ###
#from ragatouille import RAGPretrainedModel
#RAG.search("How many people live in France?")

from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM

from tqdm import tqdm
from glob import glob

**LLM Juge : Llama3**

In [ ]:
 llama3_8b = ChatOllama(model="llama3:instruct",
                   base_url = "http://10.179.0.20:5911", # Pour aller interroger l'Ollama à distance (via GPU) / Plus besoin de faire le choix des GPU en début de notebook !?
                   temperature=0.0) #local_llm

<ipython-input-21-8fcc79500578>:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llama3_8b = ChatOllama(model="llama3:instruct",


**Import QA Dataset**

We extracted a sample of 500 questions asked by real users and aanswers generated by the RAG

In [ ]:
docs = pd.read_csv('transactions_20250404 (1).csv')

**Set Up critique agent**

The questions can have many flaws. A quality check is done using critique agents that will rate each question on several criteria:
- Groundedness: can the question be answered from the given context?
- Relevance: is the question relevant to users?
- Stand-alone: is the question understandable free of any context, for someone with domain knowledge?

Each agent will received a score.
Questions with score at any agent will be dropped of the evaluation dataset.

When asking the agents to output a score, we first ask them to produce its rationale. This will help us verify scores, but most importantly, asking it to first output rationale gives the model more tokens to think and elaborate an answer before summarizing it into a single score token.

In [ ]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to a user specialized in writing commercial proposition who wishes to have answers about Thales and products developed by Thales.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independant this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like WCC, CfRx button, HCI  or TAHS and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """
